# Ejercicio 3 — Búsqueda BLAST automatizada desde fichero FASTA

En este ejercicio se desarrolla un flujo de trabajo bioinformático completo para realizar búsquedas de similitud BLAST partiendo de un fichero de entrada externo. A diferencia de los ejercicios anteriores, donde la secuencia se introducía manualmente, aquí simulamos un escenario real donde las secuencias provienen de archivos en formato **FASTA**.

El objetivo es comparar una secuencia de ADN problema contra bases de datos de nucleótidos utilizando dos estrategias:
1.  **BLAST Online:** Consultando la base de datos `nt` (nucleótidos) del NCBI a través de Internet.
2.  **BLAST Local:** Ejecutando el algoritmo sobre una base de datos personalizada construida en el propio sistema.

Finalmente, el programa debe filtrar los resultados para un **organismo específico** (en este caso, *Homo sapiens*) y calcular estadísticas clave: el número de coincidencias (*hits*) y el **E-value medio**, el cual nos servirá para evaluar la calidad global de los alineamientos encontrados.

In [1]:
from Bio.Blast import NCBIWWW, NCBIXML
from Bio.Blast.Applications import NcbiblastnCommandline
from Bio import Entrez, SeqIO
import os
import shutil
import subprocess
import numpy as np

# CONFIGURACIÓN
Entrez.email = "mariana.bordes101@alu.ulpgc.es"  # Tu email real
ARCHIVO_FASTA = "query_insulin.fasta"
DB_LOCAL_NAME = "local_db_insulina"
ORGANISMO = "Homo sapiens"

# Verificación inicial: ¿Está BLAST+ instalado?
if not shutil.which("blastn"):
    raise RuntimeError("Error Crítico: No se encuentra 'blastn' en el sistema. Instala NCBI BLAST+ y añádelo al PATH.")
print("Sistema verificado: BLAST+ está instalado correctamente.")

Sistema verificado: BLAST+ está instalado correctamente.


## 1. Configuración y lectura de secuencias
Para garantizar la reproducibilidad del experimento, el programa no solo lee un fichero FASTA, sino que primero **lo genera** asegurándose de que contiene una secuencia biológica válida.

En este caso, se utiliza un fragmento del gen de la **Insulina humana (INS)**. Trabajar con una secuencia real conocida es fundamental para obtener E-values significativos (cercanos a cero) y validar que el filtrado por organismo funciona correctamente.

Además, se verifica la presencia de las herramientas de línea de comandos (`blastn`) antes de iniciar el proceso, evitando errores de ejecución posteriores.

In [ ]:
secuencia_real = (
    "AGCCCTCCAGGACAGGCTGCATCAGAAGAGGCCATCAAGCAGGTCTGTTCCAAGGGCCTTTGCGTCAGGTGG"
    "GCTCAGGATTCCAGGGTGGCTGGACCCCAGGCCCCAGCTCTGCAGCAGGGAGGACGTGGCTGGGCTCGTGAA"
    "GCATGTGGGGGTGAGCCCAGGGGCCCCAAGGCAGGGCACCTGGCCTTCAGCCTGCCTCAGCCCTGC"
)

with open(ARCHIVO_FASTA, "w") as f:
    f.write(">gi|123456|ref|NM_000207.3| Homo sapiens insulin (INS), mRNA\n")
    f.write(secuencia_real + "\n")

print(f"Archivo '{ARCHIVO_FASTA}' generado con secuencia del gen INS.")

# Cargar para verificar
record = SeqIO.read(ARCHIVO_FASTA, "fasta")
secuencia = str(record.seq)
print(f"Longitud de la secuencia: {len(secuencia)} bp")

Archivo 'query_insulin.fasta' generado con secuencia del gen INS.
Longitud de la secuencia: 210 bp


## 2. Definición de funciones de análisis
Dado que se obtendrán resultados tanto de la búsqueda online como de la local, se define una función modular `filtrar_resultados`.

Esta función procesa los registros XML devueltos por BLAST y aplica un filtro lógico:
1.  Itera sobre todos los alineamientos encontrados.
2.  Verifica si el nombre del **organismo objetivo** aparece en la descripción del alineamiento (título).
3.  Si hay coincidencia, almacena el *E-value* (valor esperado) de ese alineamiento.

Finalmente, calcula la media aritmética de los E-values recopilados. Este estadístico es útil para resumir la "confiabilidad promedio" de los resultados obtenidos para la especie de interés.

In [3]:
def filtrar_resultados(blast_record, organismo_objetivo):
    """
    Filtra los alineamientos por organismo y calcula estadísticas.
    Retorna: (número de hits, e-value medio)
    """
    evalues = []
    
    # Recorremos todos los alineamientos
    for alignment in blast_record.alignments:
        # Buscamos el organismo en la definición del hit
        if organismo_objetivo.lower() in alignment.title.lower():
            for hsp in alignment.hsps:
                evalues.append(hsp.expect)
    
    # Cálculo seguro de la media
    if len(evalues) > 0:
        return len(evalues), np.mean(evalues)
    else:
        return 0, 0.0

## 3. Búsqueda BLASTn Online
Se utiliza la función `NCBIWWW.qblast` de Biopython para enviar la secuencia leída del fichero FASTA a los servidores del NCBI.

- **Programa:** `blastn` (búsqueda de nucleótidos contra nucleótidos).
- **Base de datos:** `nt`. Esta es la colección no redundante de nucleótidos del NCBI, que contiene millones de secuencias de todos los organismos secuenciados.
- **Filtrado:** Posteriormente, aplicamos nuestra función para quedarnos solo con los resultados de *Homo sapiens*.


In [ ]:
print(f"Iniciando BLAST online para {ORGANISMO} en base de datos 'nt'...")
print("Esto puede tardar unos minutos...")

try:
    handle = NCBIWWW.qblast(
        program="blastn",
        database="nt",
        sequence=secuencia,
        expect=0.05 # Filtro previo para velocidad
    )
    
    blast_record_online = NCBIXML.read(handle)
    print("BLAST Online finalizado con éxito.")
    
    # Procesar
    num_online, evalue_online = filtrar_resultados(blast_record_online, ORGANISMO)

    print(f"\n--- RESULTADOS BLAST ONLINE ({ORGANISMO}) ---")
    print(f"Hits encontrados: {num_online}")
    print(f"E-value medio:    {evalue_online:.4e}")
    print("---------------------------------------------")

except Exception as e:
    print(f"Error en la conexión con NCBI: {e}")

Iniciando BLAST online para Homo sapiens en base de datos 'nt'...
Esto puede tardar unos minutos...
BLAST Online finalizado con éxito.

--- RESULTADOS BLAST ONLINE (Homo sapiens) ---
Hits encontrados: 21
E-value medio:    3.0253e-78
---------------------------------------------


## 4. Búsqueda BLASTn Local y construcción de base de datos
Para realizar una búsqueda local, no basta con tener la secuencia consulta; necesitamos una **base de datos de referencia** contra la cual comparar.

En este paso, el script realiza un proceso avanzado de automatización:
1.  **Generación de la Base de Datos:** Crea un archivo FASTA local (`mis_genomas.fasta`) que incluye nuestra secuencia objetivo y secuencias de control ("ruido").
2.  **Indexado (makeblastdb):** Utiliza el módulo `subprocess` para llamar al comando `makeblastdb` del sistema. Esto convierte el FASTA en archivos binarios indexados (`.nhr`, `.nin`, `.nsq`) que BLAST puede leer eficientemente.
3.  **Ejecución (blastn):** Lanza el ejecutable localmente contra esta nueva base de datos.

Este enfoque asegura que el ejercicio sea **autocontenido**: crea su propio entorno de pruebas sin depender de archivos externos preexistentes.

In [ ]:
# --- PREPARACIÓN DEL ENTORNO LOCAL ---
import subprocess
import sys

# 1. Crear un archivo FASTA que servirá como nuestra "Base de Datos Local"
db_fasta = "mis_genomas.fasta"
with open(db_fasta, "w") as f:
    f.write(">secuencia_objetivo_local Descripción con Homo sapiens\n")
    f.write(secuencia + "\n") 
    f.write(">secuencia_random Descripción con E. coli\n")
    f.write("ATGCGTACGATCGATCGATCGATCGTTAGCATGCGTACGATCGATCGATCGATCGTTAGC\n")

# 2. Construir la base de datos BLAST (makeblastdb)
print(f"Generando base de datos local '{DB_LOCAL_NAME}'...")

cmd_makedb = f"makeblastdb -in {db_fasta} -dbtype nucl -out {DB_LOCAL_NAME}"

# Capturamos errores del sistema
try:
    proceso_db = subprocess.run(
        cmd_makedb, 
        shell=True, 
        capture_output=True, 
        text=True
    )
    
    if proceso_db.returncode != 0:
        # Si falla, imprimimos el error pero NO rompemos el programa
        print("\n[!] ERROR CRÍTICO AL CREAR LA BASE DE DATOS LOCAL:")
        print(f"Código de salida: {proceso_db.returncode}")
        print(f"Mensaje de error: {proceso_db.stderr}")
        if proceso_db.returncode == 3221225477:
            print("DIAGNÓSTICO: Error 'Access Violation'. Tu instalación de BLAST+ en Windows está corrupta.")
            print("SOLUCIÓN: Ejecuta 'conda install -c bioconda blast --force-reinstall' en tu terminal.")
        blast_local_disponible = False
    else:
        print(f"Base de datos local '{DB_LOCAL_NAME}' creada correctamente.")
        blast_local_disponible = True

except Exception as e:
    print(f"\n[!] Error inesperado al llamar al sistema: {e}")
    blast_local_disponible = False


# --- EJECUCIÓN DE BLAST LOCAL ---

if blast_local_disponible:
    xml_out_local = "resultados_local.xml"
    
    blast_cline = NcbiblastnCommandline(
        cmd="blastn",
        query=ARCHIVO_FASTA,
        db=DB_LOCAL_NAME,
        outfmt=5,
        out=xml_out_local
    )

    print(f"Ejecutando BLAST local...")
    try:
        stdout, stderr = blast_cline()
        
        with open(xml_out_local) as result_handle:
            blast_record_local = NCBIXML.read(result_handle)

        num_local, evalue_local = filtrar_resultados(blast_record_local, ORGANISMO)

        print(f"\n--- RESULTADOS BLAST LOCAL ({ORGANISMO}) ---")
        print(f"Hits encontrados: {num_local}")
        print(f"E-value medio:    {evalue_local:.4e}")
        print("--------------------------------------------")

    except Exception as e:
        print(f"Error ejecutando BLAST local: {e}")
        num_local, evalue_local = 0, 0
else:
    print("\n[AVISO] Se omite la búsqueda local debido a errores en la instalación de BLAST.")
    num_local, evalue_local = 0, 0

Generando base de datos local 'local_db_insulina'...

[!] ERROR CRÍTICO AL CREAR LA BASE DE DATOS LOCAL:
Código de salida: 3221225477
Mensaje de error: 
DIAGNÓSTICO: Error 'Access Violation'. Tu instalación de BLAST+ en Windows está corrupta.
SOLUCIÓN: Ejecuta 'conda install -c bioconda blast --force-reinstall' en tu terminal.

[AVISO] Se omite la búsqueda local debido a errores en la instalación de BLAST.


## 5. Análisis Comparativo y Conclusiones
Finalmente, se muestran y comparan las métricas obtenidas en ambos entornos.

Es esperable observar una gran diferencia en el **número de hits**:
- **Online:** Al buscar en la base de datos `nt` (grande), encontraremos múltiples entradas correspondientes a transcritos, secuencias genómicas y variantes de la insulina humana almacenadas en el NCBI.
- **Local:** Al buscar en nuestra base de datos personalizada (más limitada), solo encontraremos la secuencia exacta que nosotros mismos introdujimos.

El **E-value** también variará, ya que este valor depende estadísticamente del tamaño de la base de datos: cuanto más pequeña es la base de datos, menos probable es encontrar una coincidencia por azar, lo que puede resultar en E-values extremadamente bajos (significativos) en búsquedas locales exactas.

In [6]:
print("\n=== RESUMEN FINAL ===")
print(f"Online -> Hits: {num_online} | E-value media: {evalue_online:.2e}")
print(f"Local  -> Hits: {num_local}  | E-value media: {evalue_local:.2e}")

if num_local > 0 and num_online > 0:
    print("\nCONCLUSIÓN: El programa ha funcionado correctamente en ambos entornos.")
    print("La búsqueda local es instantánea porque usamos una base de datos reducida,")
    print("mientras que la online busca en toda la base 'nt' del NCBI.")
else:
    print("\nNOTA: Revisa si la conexión a internet falló o si los filtros son demasiado estrictos.")


=== RESUMEN FINAL ===
Online -> Hits: 21 | E-value media: 3.03e-78


NameError: name 'num_local' is not defined